# Module 6

## Performance test: Scala vs. Python

In [0]:
# create a random txt file
from random import sample
# install nltk package
from nltk.corpus import words
import nltk

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-3081098189123301> in <module> 
 1 # create a random txt file 
 2 from random import sample
 ----> 3 from nltk . corpus import words
 4 import nltk

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 156 # Import the desired module. If you’re seeing this while debugging a failed import, 
 157 # look at preceding stack frames for relevant error information. 
 --> 158 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 159 
 160 is_root_import = thread_local . _nest_level == 1 

 ModuleNotFoundError : No module named 'nltk'

In [0]:
nltk.download('words')
n = 1000
p = 300
with open("dbts:/FileStore/module6/random_text.txt", "w+") as file:
    for i in range(n):
        ligne  = " ".join(sample(words.words(), p))
        file.write(ligne)

Run the tests: PySpark vs. Scala

In [0]:
from pyspark import SparkConf, SparkContext
import time


start_time = time.time()

#conf = SparkConf().setAppName("Spark Performance Word Count")
#sc = SparkContext(conf=conf)
text_file = sc.textFile(r"dbfs:/FileStore/module6/random_text.txt")

counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)\
             .sortBy(lambda a: a[1],ascending=False)
counts.saveAsTextFile(r"dbfs:/FileStore/module6/output_python")

print( "Elapsed Time in mili seconds: " , (time.time() - start_time)*1000)

Elapsed Time in mili seconds: 13747.186660766602

In [0]:
%scala

val start1: Long = System.currentTimeMillis
val text_file = sc.parallelize(Array("dbfs:/FileStore/module6/random_text.txt"))
val count  = text_file.flatMap(line => line.split(" "))
                                        .map(word  => (word, 1))
                                        .reduceByKey(_+_)
                                        .sortBy(_._2)


count.saveAsTextFile("dbfs:/FileStore/module6/output_scala2")

val end1: Long = System.currentTimeMillis
System.out.println("Elapsed Time in mili seconds: " + (end1 - start1))

Elapsed Time in mili seconds: 845
start1: Long = 1646595471377
text_file: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[39] at parallelize at command-3081098189123344:3
count: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[47] at sortBy at command-3081098189123344:7
end1: Long = 1646595472222

Results

| PySpark | Scala |
| --- | --- |
|Elapsed Time in mili seconds:  **13747.186660766602** | Elapsed Time in mili seconds: **845** |